In [1]:
import os

In [2]:
import pandas

In [3]:
from pyiron import Project

In [4]:
pr = Project("pyiron")

In [5]:
pr.remove_jobs_silently(recursive=True)

In [6]:
structure = pr.create_ase_bulk("Cu")

In [7]:
job = pr.create_job(pr.job_type.Lammps, "lmp_runner")

In [8]:
job.structure = structure

In [9]:
pot_dir = os.path.abspath(os.path.join(os.path.curdir, "potentials", "runner"))
pot_dir

'/home/jovyan/potentials/runner'

In [10]:
for f in os.listdir(pot_dir):
    if "weights" in f and "data" not in f:
        new_weights_name = f[3:-3] + "data"
        os.rename(os.path.join(pot_dir, f), os.path.join(pot_dir, new_weights_name))

In [11]:
with open(os.path.join(pot_dir, "input.nn"), 'r') as f:
    lines = f.readlines()

lines_updated = []
for l in lines:
    if "global_symfunction_short" in l:
        lines_updated.append(l.replace("global_symfunction_short", "symfunction_short"))
    else:
        lines_updated.append(l)
lines_updated.append("\ncutoff_type 6 0.0 # Cutoff type.")

with open(os.path.join(pot_dir, "input.nn"), 'w') as f:
    f.writelines(lines_updated)

In [12]:
pot_files = [
    os.path.join(pot_dir, f)
    for f in ["input.nn", "weights.029.data", "scaling.data"]
]
pot_files

['/home/jovyan/potentials/runner/input.nn',
 '/home/jovyan/potentials/runner/weights.029.data',
 '/home/jovyan/potentials/runner/scaling.data']

In [13]:
potential = pandas.DataFrame({
    "Config": [["pair_style nnp dir './' showewsum 0\n", "pair_coeff * * 6.01\n"]], 
    "Filename": [pot_files],
    "Model": ["RuNNer"],
    "Name": ["Cu-runner"],
    "Species": [["Cu"]],
    "Citations": [{}]
})
potential

,Config,Filename,Model,Name,Species,Citations
0,"[pair_style nnp dir './' showewsum 0\n, pair_coeff * * 6.01\n]","[/home/jovyan/potentials/runner/input.nn, /home/jovyan/potentials/runner/weights.029.data, /home/jovyan/potentials/runner/scaling.data]",RuNNer,Cu-runner,[Cu],{}


In [14]:
job.potential = potential

In [15]:
job.run()

2021-02-26 19:35:25,726 - pyiron_log - WARNING - Job aborted
2021-02-26 19:35:25,729 - pyiron_log - WARNING - LAMMPS (10 Feb 2021)
Reading data file ...
  triclinic box = (0.0000000 0.0000000 0.0000000) to (2.5526555 2.2106645 2.0842345) with tilt (-1.2763277 1.2763277 0.73688816)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  1 atoms
  read_data CPU = 0.001 seconds

*******************************************************************************

WELCOME TO n²p², A SOFTWARE PACKAGE FOR NEURAL NETWORK POTENTIALS!
------------------------------------------------------------------

n²p² version      : 553e609
------------------------------------------------------------
Git branch        : HEAD
Git revision      : 553e6097553afe442843cc45bce5c974b1ec2f92
Compile date/time : Feb 15 2021 12:43:35
------------------------------------------------------------

Please cite the following papers when publishing results obtained with n²p²:
---------------------------------------------------

The job lmp_runner was saved and received the ID: 1


RuntimeError: Job aborted

In [ ]:
pr.job_table()